In [ ]:
class lstm_double(nn.Module):
    
    def __init__(self, input_size, hidden_units, layers_num, lin, dropout_prob=0):
        # Call the parent init function (required!)
        super().__init__()
        # Define recurrent layer
        self.lstm = nn.LSTM(input_size=input_size, 
                            hidden_size=hidden_units,
                            num_layers=layers_num,
                            dropout=dropout_prob,
                            batch_first=True)
        
        self.norm = nn.BatchNorm1d(hidden_units)
        
        self.narr = nn.Sequential(nn.Linear(hidden_units, lin),
                                  nn.ReLU(),
                                  nn.Dropout(dropout_prob/3),
                                  nn.BatchNorm1d(lin),
                                  nn.Linear(lin, input_size)
                                 )
        self.speech = nn.Sequential(nn.Linear(hidden_units, lin),
                                    nn.ReLU(),
                                    nn.Dropout(dropout_prob/3),
                                    nn.BatchNorm1d(lin),
                                    nn.Linear(lin, input_size)
                                   )

        
    def forward(self, x, state=None, in_speech=None):
        
        x_rnn, state = self.lstm(x, state)
        
        x_rnn = x_rnn[:,-1,:]
        x_flat = self.norm(x_rnn)
        
        out_speech = self.speech(x_flat)
        out_narr = self.narr(x_flat)
        
        out_speech[in_speech==0] = 0
        out_narr[in_speech==1] = 0
        
        out = out_speech + out_narr
        
        return out, state
    
    def save(self):
        torch.save(self.state_dict(), 'double_drop_2/ckpt/best_4.ckpt')
    